In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# 初始化瀏覽器
driver = webdriver.Chrome()

# 開啟Twitter
driver.get("https://twitter.com/login")



In [2]:
username = driver.find_element(By.XPATH,'//input[@autocomplete="username"]')
username.send_keys("帳號")
username.send_keys(Keys.RETURN)

In [5]:
passward = driver.find_element(By.XPATH,'//input[@autocomplete="current-password"]')
passward.send_keys("密碼")
passward.send_keys(Keys.RETURN)

In [237]:
start_date = '2024-05-24' # 起始日期
end_date = '2024-05-25' # 結束日期
query = 'crypto' # 查詢字串
url = f'https://x.com/search?&f=top&q={query}%20lang%3Aen%20until%3A{end_date}%20since%3A{start_date}&src=typed_query'
driver.get(url)

In [214]:
# driver.find_elements(By.XPATH, "//div[contains(@aria-label, 'replies') or contains(@aria-label, 'reposts') or contains(@aria-label, 'likes') or contains(@aria-label, 'bookmarks') or contains(@aria-label, 'views')]")[8].get_attribute('aria-label')


'2 likes, 215 views'

In [236]:
"""文章 aria-label : "replies", "reposts", "likes", "bookmarks", "views"""
"""還沒完成"""


# tweet = driver.find_elements(By.XPATH, "//article[@data-testid='tweet']")
# data = []
# for i in range(len(tweet)):
#     text = tweet[i].text
#     aria_label = tweet[i].find_elements(By.XPATH, "//div[contains(@aria-label, 'views')]")[i].get_attribute('aria-label') 
#     data.append({'text': text, 'aria_label': aria_label})
# 
# df = pd.DataFrame(data)
# df

,text,aria_label
0,Jhonwick\n@Jhonwic65\n·\nMay 25\nTop Trending ...,42 views
1,SuitedReaper\n@SuitedReaper11\n·\nMay 25\n“Wha...,"2 replies, 3 reposts, 6 likes, 291 views"
2,pinkapewife\n@pinkapewife\n·\nMay 25\nit’s rea...,"1 reply, 1 repost, 12 likes, 266 views"
3,CryptoKingdomZ\n@CryptoKingdomZ\n·\nMay 25\nHo...,9 views
4,CryptoKingdomZ\n@CryptoKingdomZ\n·\nMay 25\nUS...,"1 reply, 85 views"
5,Crypto Ivar\n@CryptoIvarr\n·\nMay 25\nYou can ...,"2 reposts, 9 likes, 233 views"
6,Oversized Moose With Socks (xlmoose.eth)\n@und...,"10 likes, 149 views"
7,z3ro\n@Zer0Admi\n·\nMay 25\nthis just keep get...,"13 replies, 6 reposts, 124 likes, 19 bookmarks..."
8,"Trade Alerts, Trade Ideas and Crypto\n@AlertTr...","1 repost, 3 likes, 532 views"
9,Flo M.\n@flodjco\n·\nMay 25\nTURBO! https://yo...,"1 like, 33 views"


In [238]:
tweets = []
processed_tweet_texts = set()
tweets_data = {}
last_height = driver.execute_script("return document.body.scrollHeight")

In [239]:
def click_show_more(driver, tweet_article, tweet_list, processed_tweet_texts):
    try:
        try:    
            tweet_click = tweet_article.find_element(By.XPATH, ".//div[@data-testid='tweetText']")
            tweet_click.click()
        except Exception as e:
            print(f"Error clicking show more: {tweet_article.text}, Exception: {e}")
            try:        
                tweet_click = tweet_article.find_element(By.XPATH, ".//div[@data-testid='tweet-text-show-more-link']")
                tweet_click.click()
            except Exception as e:
                print(f"Error clicking show more: {tweet_article.text}, Exception: {e}")
                return
                
#/html/body/div[1]/div/div/div[2]/main/div/div/div/div/div/div[3]/section/div/div/div[6]/div/div/article
#/html/body/div[1]/div/div/div[2]/main/div/div/div/div/div/div[3]/section/div/div/div[6]/div/div/article/div/div/div[2]/div[2]/div[2]/div[1]
        time.sleep(3)
        tweet_text = driver.find_element(By.XPATH, "//article[@data-testid='tweet']").text
        tweet_list.append(tweet_text)
        processed_tweet_texts.add(tweet_text)
        driver.back()
        time.sleep(3)
    except Exception as e:
        print(f"Error clicking show more: {tweet_article.text}, Exception: {e}")

while True:
    tweets_articles = driver.find_elements(By.XPATH, "//article[@data-testid='tweet']")
    for tweet_article in tweets_articles:
        try:
            tweet_text = tweet_article.text
            if tweet_text not in processed_tweet_texts:
                try:
                    if "Show more" in tweet_article.text:
                        click_show_more(driver, tweet_article, tweets, processed_tweet_texts)
                    else:
                        tweets.append(tweet_article.text)
                        processed_tweet_texts.add(tweet_text)
                except Exception as e:
                    print(f"Error: processing tweet article: {e}")
                    continue
        except Exception as e:
            print(f"Error: processing tweet article: {e}")
            continue
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)  # 等待頁面加載
    # 記錄當前高度並與之前高度比較，如果沒有變化則停止滾動
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
        
    

Error clicking show more: Wadi The Crypto Hunter
@WadiCryptoHunt
·
May 25
If you haven't gone, take a look at this project. Do your own research before investing into anything.
Humble Tree
@HBThumbletree
·
May 23
Check out #HTREE $HTREE 

CA: Csd83TN18Nexx1hKhs5R52xtBsXuNWD6GUVERnS25tdm

Dex :  https://dexscreener.com/solana/e84euvzctid8skh1d6gws4ooqdwmtv44zho3gmguhtwt…
Show more
2
324, Exception: Message: element click intercepted: Element <div dir="auto" lang="en" class="css-146c3p1 r-8akbws r-krxsd3 r-dnmrzs r-1udh08x r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-bnwqim" id="id__z33soykvvyt" data-testid="tweetText" style="-webkit-line-clamp: 10; text-overflow: unset; color: rgb(231, 233, 234);">...</div> is not clickable at point (363, 20). Other element would receive the click: <input autocapitalize="sentences" autocomplete="off" autocorrect="off" placeholder="Search" spellcheck="false" enterkeyhint="search" type="text" dir="auto" aria-activedescendant="typeahe

In [240]:
len(tweets) # 這是我們爬取的推文數量

34

In [241]:
for tweet in tweets:
    print("="*50)
    print(tweet)
#每一筆資料都是str型態

Daniel Samuel Etim
@danielsamuel_et
·
May 25
Step up your trading game with 
@BHE_Exchange
! Dive into a world of endless opportunities where you call the shots. Join us in shaping the future of trading. Let's soar together. #BHEExchange #Crypto 
4
7
125
Crypto Jon
@CryptoJon__
·
May 25
Im jealous 
19
37
272
7.4K
Poké Calls
@PokeCallsCrypto
·
May 25
Replying to 
@PokeCallsCrypto
$MYC up more than 20% from call, huge pumpamentals on this big cap coin with fat liquidity  

There's only 0.5% supply of $MYC left on the Uniswap pool now 
11
19
799
9 Lives
@9livessolana
·
May 25
“We are here to stay” $9LIVES 
$sol #9LIVES #allcatsgotoheaven #memecoin #Crypto #solana
3
7
164
The Women of Crypto
@women0fCrypto
·
May 25
"Diamond" Glenn McCarthy, the original oil wildcatter, would be intrigued by today's energy shift. Meet the new wildcatters chasing natural hydrogena game-changer in clean https://independent.co.uk/climate-change/news/clean-energy-search-white-hydrogen-b2450463.html…
22
Te Ariki

In [8]:
# 將爬取的推文存儲到DataFrame中
df = pd.DataFrame(tweets, columns=["artDate", "artContent"])
df['dataSource'] = 'Twitter'
# 打印DataFrame
print(df)

                     artDate  \
0   2024-05-24T07:22:00.000Z   
1   2024-05-24T19:06:45.000Z   
2   2024-05-24T15:27:06.000Z   
3   2024-05-24T09:58:10.000Z   
4   2024-05-23T17:43:07.000Z   
..                       ...   
93  2024-05-23T19:12:00.000Z   
94  2024-05-15T05:57:06.000Z   
95  2024-05-24T09:07:38.000Z   
96  2024-05-24T20:29:48.000Z   
97  2024-05-24T21:07:00.000Z   

                                           artContent dataSource  
0                         #BITCOIN BULL FLAG RETEST!!    Twitter  
1   We have been able to determine the past 2 true...    Twitter  
2    Gifted me a little something  \n\nAlhamdulillah     Twitter  
3            GM \n\n“Turn paper into crypto.” - Jesus    Twitter  
4   When governments try take your crypto, make yo...    Twitter  
..                                                ...        ...  
93  Degen Smerf ($SMERF) - Based team, strong futu...    Twitter  
94                  C4B is +1 Today\n\nThank you all.    Twitter  
95   Top The

In [9]:
date_range = f'{start_date} to {end_date}'
# 保存DataFrame到CSV文件
df.to_csv(f"{date_range}tweets1.csv", index=False)